In [ ]:
# リスト4.6.1
# 資格情報の設定 (個別に設定します)

# ▼このスクリプトの目的
# - IBM Watson Discovery（V1）に接続するための最小コードです。
# - 認証（IAM）→ クライアント生成 → エンドポイント設定、までを行います。
# - ※本番運用では資格情報を直書きせず、環境変数やシークレットマネージャから読み込むことを推奨します。

discovery_credentials = {
    "apikey": "xxxx",  # 必須：IAM 認証用の API Key。**直書きは避け、環境変数から取得することを推奨**
    "iam_apikey_description": "xxxx",  # 参考：API Key の説明文字列（管理用途）。SDK の認証には未使用
    "iam_apikey_name": "xxxx",  # 参考：API Key の名称（管理用途）。SDK の認証には未使用
    "iam_role_crn": "xxxx",  # 参考：IAM ロールのCRN（管理用途）。SDK の認証には未使用
    "iam_serviceid_crn": "xxxx",  # 参考：サービスIDのCRN（管理用途）。SDK の認証には未使用
    "url": "xxxx",  # 必須：Discovery サービスのベースURL（例: https://api.jp-tok.discovery.watson.cloud.ibm.com）
}

# ▼補足（セキュリティの観点）：
# - 上記ディクショナリは学習・検証用の最小例です。
# - 実運用では、python-dotenv 等で .env から読み取り、os.environ 経由で参照してください。
#   例）
#     from dotenv import load_dotenv
#     load_dotenv()
#     APIKEY = os.getenv("WATSON_DISCOVERY_APIKEY")
#     URL    = os.getenv("WATSON_DISCOVERY_URL")

# Discovery APIの初期化

import json  # 後続の応答を JSON 文字列化（整形表示）したい場合に使用
import os  # 環境変数から資格情報を読む場合に使用
from ibm_watson import DiscoveryV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# ▼DiscoveryV1 のバージョン指定
# - DiscoveryV1 クライアントのコンストラクタで必須。
# - サーバ側APIの互換性ポリシーに基づく「日付文字列」で、サポートされるAPIスキーマを選びます。
# - IBMのドキュメントで利用可能な最新日付を確認し、プロジェクトで固定（ピン留め）するのが安全です。
version = "2019-04-30"

# ▼IAM 認証オブジェクトの生成
# - API Key を用いた認証方式を選択します。
# - 例外（無効キー／ネットワークエラー等）に備え、実運用では try/except を推奨します。
authenticator = IAMAuthenticator(discovery_credentials["apikey"])

# ▼Discovery クライアントの生成
# - version と authenticator を渡して初期化します。
# - この段階ではまだエンドポイント（URL）は未設定です。
discovery = DiscoveryV1(version=version, authenticator=authenticator)

# ▼サービスURLの設定
# - 利用するリージョンに応じたエンドポイントを指定します。
# - リージョンをまたいだ利用はレイテンシやデータ所在の観点から非推奨です。学習/推論でリージョンを揃えるのが基本方針です。
discovery.set_service_url(discovery_credentials["url"])

# ▼この後の典型的な呼び出し例（参考）：
# - discovery.list_environments().get_result()
# - discovery.list_collections(environment_id=..., configuration_id=...).get_result()
# - 上記の戻り値を json.dumps(..., ensure_ascii=False, indent=2) で整形出力すると可読性が上がります。
# - 429（レート制限）や5xxに備え、指数バックオフ付きのリトライ実装を用意すると運用が安定します。

In [ ]:
# リスト 4.6.2
# environment_id、collection_id、configuration_id の取得
# すでに UI で 1 つ以上の private collection（収集済みコレクション）が作成されていることが前提

# ▼環境(Environment)とは
#   Discovery V1 における論理的な作業空間。UI で通常「デフォルト環境（非 system）」が 1 つ作られている想定。
#   list_environments() は複数返す可能性があり、'system' は読み取り専用のシステム環境なので除外するのが通例。
#   ここでは先頭要素を取り、もし 'system' なら 2 番目を使うという簡便な選択を行っている。
environments = discovery.list_environments().get_result()[
    "environments"
]  # API 応答(JSON)→dict へ変換後、'environments' 配列を取得
environment_id = environments[0]["environment_id"]  # 先頭要素の environment_id を採用
if (
    environment_id == "system"
):  # 先頭がシステム環境の場合は次の要素を採用（UI で作成した環境を想定）
    environment_id = environments[1]["environment_id"]
print(
    "environment_id: ", environment_id
)  # デバッグ/記録用に ID を表示（名前も必要なら label を併せて出力）

# ▼コレクション(Collection)とは
#   収集・インデックス化されたドキュメントの集合。クエリの対象単位。
#   list_collections(environment_id) は該当環境内のコレクション一覧を返す。
#   ここでは「最初のコレクション 1 件」を仮定して先頭を選ぶ（複数ある場合は名前や目的で選択するのがより安全）。
collection_id = discovery.list_collections(environment_id).get_result()["collections"][
    0
]["collection_id"]
print("collection_id: ", collection_id)

# ▼コンフィグ(Configuration)とは
#   取り込み時の前処理（コンバータ/正規化/アノテーション抽出など）の定義。
#   通常はデフォルトをベースに UI で調整したものを利用。ここでも先頭要素を取得している。
configuration_id = discovery.list_configurations(environment_id).get_result()[
    "configurations"
][0]["configuration_id"]
print("configuration_id: ", configuration_id)

# （補足）
# - 本コードは「先頭要素を選ぶ」前提の簡易版。実運用では:
#   * 'label'（環境名/コレクション名/設定名）を列挙して、人間可読名でフィルタ選択する。
#   * try/except で IndexError/KeyError を捕捉し、「環境が存在しない」「コレクション未作成」等を明示する。
#   * 日本語名を print する場合は json.dumps(..., ensure_ascii=False) を使うと文字化けを避けられる。

In [ ]:
# リスト 4.6.3
# 文書ロード関数
# 前提:
#  - すでに DiscoveryV1 のクライアント `discovery`、`environment_id` が作成済みであること
#  - `json`, `os`, `time` が import 済みであること（本関数内で time.sleep を使用）
#  - Knowledge Studio で作成・学習したモデルに投入するための素性データや原文を「1 item = 1 文書」として投入する想定
#
# 役割:
#  - 渡された配列 `sample_data`（各要素は JSON/dict）を 1 件ずつ一時 JSON ファイルに書き出し、
#    Discovery のコレクションへ add_document API を用いて登録する。
#  - コレクション内の「現在処理中(processing)の文書数」を見ながら、バックプレッシャを簡易的にかける。


# collection_id: 対象コレクション ID（Discovery のコレクション）
# sample_data: 書き込み対象テキスト (dict の配列: 各要素が 1 文書)
# key_name: 文書のユニークキー名称（各 item からファイル名を作るために使用）
def load_text(collection_id, sample_data, key_name):
    for item in sample_data:
        # 1) アイテム確認（デバッグ/ログ用途）
        print(item)

        # 2) ファイル名の決定
        #    - ユニークキー（例: 'app_id' や 'id'）をファイル名に利用する。
        #    - str() しておくことで、数値キーでも安全にファイル名化できる。
        key = item.get(key_name)
        filename = str(key) + ".json"

        # 3) 一時 JSON ファイルへ書き出し
        #    - 文字化けを避けるなら encoding='utf-8' 指定が望ましい。
        #    - ensure_ascii=False で日本語を可読のまま保存可能（ここでは API 側に渡すだけなので必須ではない）。
        f = open(filename, "w")
        json.dump(item, f)
        f.close()

        # 4) バックプレッシャ制御（簡易）
        #    - 現在「処理中(processing)」の文書数が閾値以上なら待機する。
        #    - ここでは 20 を閾値にしており、投入スパイクでの失敗/スロットルを回避する意図。
        collection = discovery.get_collection(
            environment_id, collection_id
        ).get_result()
        proc_docs = collection["document_counts"]["processing"]
        while True:
            if proc_docs < 20:
                break
            print("busy. waiting..")
            time.sleep(10)
            # ★重要: 再取得時も .get_result() が必要（ないと DetailedResponse のままで dict として参照できない）
            collection = discovery.get_collection(
                environment_id, collection_id
            ).get_result()
            proc_docs = collection["document_counts"]["processing"]

        # 5) Discovery へドキュメント追加
        #    - add_document はファイルハンドルを受け取り、取り込み/コンバート/インデックスを行う。
        #    - 必要に応じて metadata= でメタデータ（id/title/type など）を付加できる。
        #    - 実運用では戻り値を .get_result() で受けて status/notice を確認するのが望ましい。
        with open(filename) as f:
            add_doc = discovery.add_document(environment_id, collection_id, file=f)
            # _ = add_doc.get_result()  # ← 必要であれば応答 JSON を確認

        # 6) 後始末: 一時ファイルを削除（ローカルに不要なゴミを残さない）
        os.remove(filename)

        # （補足）
        # - 1 件ずつファイル化→投入のため I/O が多くなる。大量投入では tempfile.NamedTemporaryFile や
        #   メモリバッファ(BytesIO)の利用、あるいは Discovery のバッチ投入の検討が有効。
        # - ファイル名の衝突回避や OS 非対応文字（/ など）対策として、key をサニタイズするのが安全。
        # - 例外（ネットワークエラー/429/5xx）に備えて try/except とリトライ・指数バックオフを組み込むと堅牢化できる。

In [ ]:
#  リスト 4.6.4
# 文書ロードサンプル

# ロードテスト用テキスト
sample_data = [
    {"app_id": 1, "title": "最初のテキスト", "text": "サンプルテキストその1。"},
    {"app_id": 2, "title": "2番目のテキスト", "text": "新幹線はやぶさが好きです。"},
    {"app_id": 3, "title": "3番目のテキスト", "text": "令和元年に転職しました。"},
]

# 文書ロードテスト
load_text(collection_id, sample_data, "app_id")

In [ ]:
# リスト4.6.5
# 特定のコレクションの全文書を削除する関数
# collection_id: 対象コレクション


def delete_all_docs(collection_id):
    """
    Discovery のコレクション内に「利用可能(available)」として計上されている
    すべての文書を削除するユーティリティ関数。

    注意/前提:
      - 事前に `discovery` クライアントと `environment_id` が有効に初期化されていること。
      - ここで削除対象として数えるのは `available` 文書のみ。取り込み中/処理中(processing)や
        エラー(FAILED)の文書は件数に含まれないため、完全削除を厳密に行う場合は
        追加の状態確認や再試行が必要になりうる。
      - `query(..., count=doc_count)` は API 側の上限に依存する（大量件数ではページングが必要）。
        件数が多いコレクションでは、`offset` を使った繰り返し取得に書き換えることを推奨。
      - 連続削除は API レートリミットに抵触する場合がある。必要に応じて `time.sleep(...)`
        を挟むなどのスロットリングを検討すること。
    """

    # 1) コレクションのメタ情報を取得し、現在「利用可能(available)」な文書件数を取得
    collection = discovery.get_collection(environment_id, collection_id).get_result()
    doc_count = collection["document_counts"]["available"]

    # 2) 利用可能な文書をすべて列挙（id のみ返すクエリ）
    #    - `return_fields='id'` により転送データ量を最小化
    #    - 大規模コレクションでは count の上限に注意（必要に応じてページングに変更）
    results = discovery.query(
        environment_id, collection_id, return_fields="id", count=doc_count
    ).get_result()["results"]

    # 3) 各文書の id を取り出し
    ids = [item["id"] for item in results]

    # 4) 1 件ずつ削除
    #    - 大量削除時は try/except によるリトライ/バックオフを推奨
    #    - 処理中ドキュメントはこのループに載らない可能性がある点に注意
    for id in ids:
        print("deleting doc: id =" + id)
        discovery.delete_document(environment_id, collection_id, id)

In [ ]:
# リスト4.6.6

# 全件削除テスト
delete_all_docs(collection_id)

In [ ]:
# リスト4.6.7
#
# 検索用関数
# - Watson Discovery (V1) のクエリ API を用いて、指定コレクションから文書を取得するユーティリティ。
# - 本関数は「現在利用可能(available)」な文書総数を先に取得し、その件数を上限として検索結果を引き当てる。
#   大量件数の場合や API 上限がある場合はページング（offset を用いた繰り返し取得）への拡張を推奨。
#
# 前提:
#   - 事前に `discovery` クライアントおよび `environment_id` が有効に初期化されていること。
#   - `collection_id` は対象コレクションの ID（UI や API で取得済みの文字列）。
#
# 引数:
#   - collection_id (str): 検索対象コレクションの ID。
#   - query_text  (str):  Discovery V1 のクエリ構文で記述された検索条件式。
#                         例) 'title::"温泉" AND enriched_text.entities.text:"硫黄泉"'
#                         空文字列/None の場合、スコア順の全件取得となる（件数が多いと負荷増大に注意）。
#   - return_fields (str | list[str]): 返却したいフィールド名。カンマ区切りの文字列または配列。
#                                      転送量削減のため必要最小限のフィールドのみ指定するのが実務上は望ましい。
#
# 戻り値:
#   - list[dict]: `results` 配列そのものを返す。各要素はヒット文書のメタ/本文フィールドを含むディクショナリ。
#
# 実装メモ/注意点:
#   - 件数の多いコレクションで `count=doc_count` とすると API 側の最大件数制限にかかる可能性がある。
#     その場合は `count` を安全な上限（例: 1000）に設定し、`offset` を増やしながら複数回クエリする。
#   - 並行実行や連続呼び出しではレートリミット(429)に注意。必要に応じてリトライ/バックオフを実装。
#   - フィールド指定はスキーマ（コレクションの構成）に依存。誤ったフィールド名は無視される/エラーとなる。
#   - Discovery の V1 は将来的に非推奨となる可能性があるため、V2 (Watson Discovery) への移行設計も検討。
#
def query_documents(collection_id, query_text, return_fields):
    # 1) コレクションの状態を取得し、現在「利用可能(available)」な文書件数を把握
    #    ここで表示している件数は、クエリのヒット件数ではなくコレクション全体の利用可能件数である点に注意。
    collection = discovery.get_collection(environment_id, collection_id).get_result()
    doc_count = collection["document_counts"]["available"]
    print("doc_count: ", doc_count)

    # 2) クエリを実行
    #    - `query` には Discovery V1 のクエリ構文を渡す。
    #    - `count` に doc_count をそのまま設定しているため、件数が非常に多い場合は API 上限に注意。
    #      その場合はページング（count を小さく、offset を増やすループ）に置き換えること。
    #    - `return_fields` は転送量削減のため可能な限り絞り込む。
    query_results = discovery.query(
        environment_id,
        collection_id,
        query=query_text,
        count=doc_count,
        return_fields=return_fields,
    ).get_result()["results"]

    # 3) 検索結果（各文書の辞書オブジェクトの配列）を返却
    return query_results

In [ ]:
# リスト4.6.8
# 「サンプル」をキーとした検索
#
# 目的:
#   - Watson Discovery (V1) のクエリ DSL を用い、フィールド `text` に
#     日本語キーワード「サンプル」を含む文書をコレクション全体から取得する。
#
# 背景と理論的補足:
#   - Discovery V1 の `query` 引数は “フィールド指定付きのブール/語彙検索 DSL” を受け付ける。
#     ここでは `text:サンプル` により、取り込んだ原文フィールド（本チュートリアルでは JSON の `text`）に
#     「サンプル」というトークン（形態素/語）が含まれる文書を検索する。
#   - 同 API には自然文での検索 `natural_language_query` もあるが、本例は厳密なフィールド指定を行いたいので `query` を採用。
#   - 日本語の一致はコレクションの構成（使用中の Configuration におけるトークナイズ/正規化/言語判定など）に依存する。
#     形態素分割や正規化が有効な設定であれば、語形の揺れにある程度ロバストになる。
#   - 厳密一致/フレーズ検索をしたい場合は引用符で囲む（例: text:"サンプル データ"）。
#     前方一致等の曖昧化にはワイルドカード（例: text:サンプ*）が使えるが、負荷やノイズ増加に注意。
#
# ベストプラクティス:
#   - `return_fields` を明示的に絞ると、転送量とレンダリングコストを抑えられる（本例は app_id, title, text）。
#   - 大規模コレクションでは、query_documents 側の `count` を安全値に抑え、`offset` でページングする設計に拡張する。
#   - 0件ヒット時のハンドリング（代替クエリ、サジェスト、サマリー提示など）を用意すると UX が向上する。
#
# 事前条件:
#   - `discovery`, `environment_id`, `collection_id` の初期化が済んでいる（4.6.1～4.6.2 参照）。
#   - `query_documents` ユーティリティ関数が定義済み（4.6.7 参照）。

# 1) クエリ式の定義:
#    - `text` フィールドに「サンプル」を含む文書を対象とする。
query_text = "text:サンプル"

# 2) 返却フィールドの指定:
#    - app_id と title は識別・表示用、text は本文確認用に返す。
#      実務では UI 都合に合わせて必要最小限に絞ること（大規模本文はハイライト断片を返すのが一般的）。
return_fields = "app_id,title,text"

# 3) 検索実行:
#    - `query_documents` はコレクションの available 件数を上限に結果を取得する簡易版。
#      取得上限やページングは要件に応じて調整する。
query_results = query_documents(collection_id, query_text, return_fields)

# 4) 結果の整形出力:
#    - ensure_ascii=False を指定して日本語を可読表示。
#    - ダンプ前に件数や上位数件のみを表示するなど、可視性配慮も検討。
import json

print(json.dumps(query_results, indent=2, ensure_ascii=False))

In [ ]:
# リスト4.6.9
# 「はやぶさ」をキーとした検索
#
# 目的:
#   - Watson Discovery のクエリ DSL を用い、取り込んだ文書のうち
#     原文フィールド `text` に「はやぶさ」を含むものを列挙する。
#
# 背景と理論的補足:
#   - `query` 引数はフィールド指定検索を受け付ける。`text:はやぶさ` は
#     「フィールド `text` にトークン 'はやぶさ' が存在する文書」を意味する。
#   - 日本語の一致は、コレクションの Configuration（トークナイズ/正規化/言語処理）が前提。
#     形態素解析の粒度や記号正規化の方針により、語の切れ方や一致の挙動が変わる。
#   - 曖昧性への注意:
#       * 「はやぶさ」は一般名詞（隼）/ 固有名詞（探査機）/ 列車名（新幹線はやぶさ）など複数義がある。
#         目的が限定的なら、フィールド追加やブール結合（例: `text:はやぶさ AND text:新幹線`）で文脈を絞る。
#       * 厳密な句一致は引用符で囲む（例: `text:"新幹線 はやぶさ"`）。ただし語順固定や記号の扱いは設定依存。
#       * 前方/部分一致が必要ならワイルドカード（例: `text:はやぶ*`）もあるが、ノイズ増加・負荷に注意。
#   - ランキングは一般に語の希少性（逆文書頻度）や出現頻度、マッチ位置等を考慮した関連度スコアで並ぶ。
#     長文本文を返す場合は `highlight` を併用し、ヒット周辺の断片を併せて表示すると可読性が高い。
#
# ベストプラクティス:
#   - `return_fields` は必要最小限に。UI では本文全文ではなく断片を提示する設計にする。
#   - 大規模コレクションではページング（`offset`/`count`）を実装し、タイムアウトや上限超過を避ける。
#   - 0 件時の代替クエリ（同義語、かな/漢字ゆれ、別表記）をサジェストすると検索体験が向上。
#
# 事前条件:
#   - `discovery`, `environment_id`, `collection_id` の初期化が完了している（4.6.1～4.6.2 参照）。
#   - `query_documents` ユーティリティ関数が定義済み（4.6.7 参照）。
#   - `json` は前セルで `import json` 済み（未インポートなら本セル冒頭で import する）。

# 1) クエリ式の定義:
#    - `text` フィールドに「はやぶさ」を含む文書を対象とする。
query_text = "text:はやぶさ"

# 2) 返却フィールドの指定:
#    - app_id/title は識別用、text は本文確認用。
#      本番はハイライト断片や別フィールド（要約/見出し）に差し替えるのが一般的。
return_fields = "app_id,title,text"

# 3) 検索実行:
#    - コレクションの available 件数を上限に取得する簡易実装。
#      実運用では上限・ページング・タイムアウト再試行などを拡張。
query_results = query_documents(collection_id, query_text, return_fields)

# 4) 結果の整形出力:
#    - ensure_ascii=False で日本語を可読表示。
#    - 大量出力の場合は先頭 N 件に制限、あるいはタブular 表示にする。
print(json.dumps(query_results, indent=2, ensure_ascii=False))

In [ ]:
# リスト 4.6.10
# 形態素辞書の定義例

custom_list = [
    {
        "text": "はやぶさ",
        "tokens": ["はやぶさ"],
        "readings": ["ハヤブサ"],
        "part_of_speech": "カスタム名詞",
    }
]

In [ ]:
# リスト4.6.11
# 形態素辞書の登録用関数（Watson Discovery V1）


def register_tokenization_dictionary(collection_id, tokenization_rules):
    """
    目的:
        IBM Watson Discovery のコレクションに「形態素辞書（Tokenization Dictionary）」を登録し、
        登録ジョブが完了するまでステータスをポーリングする。

    前提:
        - 事前に DiscoveryV1 クライアント `discovery`、`environment_id` が初期化済みであること。
        - `collection_id` は登録先のコレクション ID。
        - `tokenization_rules` は辞書定義の JSON。日本語等の形態素切り出しを上書きしたい語や
          固有表記を列挙する。
          例（概念スキーマの一例。実際のキー名・構造は利用する API バージョンの公式ドキュメントに従う）:
              tokenization_rules = {
                  "tokenization_rules": [
                      {
                          "text": "はやぶさ",      # 原表記（コーパス内の文字列）
                          "tokens": ["はやぶさ"]   # 望ましいトークン列（分割/正規化の上書き）
                      },
                      {
                          "text": "カルシウム・ナトリウム－硫酸塩・塩化物泉",
                          "tokens": ["カルシウム", "ナトリウム", "硫酸塩", "塩化物泉"]
                      }
                  ]
              }

    理論的背景:
        - Tokenization Dictionary は、インデクシング/検索時のトークン化（語の切り出し）を
          ユーザ定義で補正する仕組み。一般辞書に載らない専門語や複合語（温泉の泉質名、
          鉱物名、地名の連接など）を検索要件に沿って安定に切り出すことで、再現率/適合率を
          改善できる。
        - 「切り過ぎ（過分割）」や「繋ぎ過ぎ（未分割）」は、TF・IDF や BM25、構造化抽出の
          上流品質に直結する。固有表現の粒度を安定化させることは、同義語展開・クエリ拡張・
          ハイライトにも効く。

    注意点:
        - 本関数はジョブ完了まで busy-wait（10 秒間隔）する。実運用ではタイムアウトや
          最大試行回数、指数バックオフ、例外処理を追加すること。
        - 登録が成功しても、既存ドキュメントの再インデックスが必要になる場合がある。
          品質評価前に小規模データで A/B 検証すること。
        - ステータス値（例: 'pending' → 完了状態）は Discovery のバージョンで異なることがある。
          公式ドキュメントの最新仕様を参照のこと。

    引数:
        collection_id (str): 対象コレクション ID
        tokenization_rules (dict): トークナイズ辞書の JSON ペイロード

    戻り値:
        なし（進捗を print で出力）。必要に応じて最終ステータスを返すよう拡張可能。
    """
    # 1) 形態素辞書の登録をリクエスト
    #    - 大きな辞書や多エントリの場合はサーバ側で非同期処理になる。
    res = discovery.create_tokenization_dictionary(
        environment_id, collection_id, tokenization_rules=tokenization_rules
    )

    # 2) 登録ジョブの進行状況をポーリング
    #    - 初回呼び出しで現在状態を取得し、'pending' の間は 10 秒スリープして再確認。
    #    - 無限ループ回避のため、本番はタイムアウト/最大リトライ回数を設けること。
    import time  # 可能ならファイル冒頭で import する（ここでは関数内に限定）

    res = discovery.get_tokenization_dictionary_status(
        environment_id, collection_id
    ).get_result()

    while res.get("status") == "pending":
        time.sleep(10)
        res = discovery.get_tokenization_dictionary_status(
            environment_id, collection_id
        ).get_result()
        # 進捗を標準出力へ（運用ではロガー推奨）
        print(res)

    # 3) 最終状態（成功/失敗）を確認して必要に応じてハンドリングする余地を残す。
    # print('final status:', res)  # デバッグ用途

In [ ]:
# リスト4.6.12
# 形態素辞書登録用関数の呼び出し例

register_tokenization_dictionary(collection_id, custom_list)

In [ ]:
# リスト 4.6.13
# 「はやぶさ」で検索できることの確認

delete_all_docs(collection_id)
load_text(collection_id, sample_data, "app_id")

import time

time.sleep(30)

query_text = "text:はやぶさ"
return_fields = "app_id,title,text"
query_results = query_documents(collection_id, query_text, return_fields)

print(json.dumps(query_results, indent=2, ensure_ascii=False))